In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen


import seaborn as sns

%matplotlib inline

In [2]:
df = pd.read_csv('../input/2012_qb_weeks1_8.csv')
df1=pd.read_csv('../input/2012_qb_weeks_9_16.csv')
df3 = pd.read_csv('../input/2012_qb_week17.csv')

In [3]:
raw_qb_2012 = pd.concat([df,df1], axis=0)
raw_qb_2012 = pd.concat([raw_qb_2012, df3], axis=0)
raw_qb_2012.drop('Rk', axis =1, inplace = True)
raw_qb_2012.shape

(629, 18)

In [4]:
raw_qb_2012.head()

,Player,Pos,Week,Team,Opp,Comp,Att,Pct,Yds,Yds/Att,TD,Int,QB Rating,Att.1,Yds.1,Yds/Att.1,TD.1,Fantasy Points
0,Matt Ryan,QB,1,ATL,KC,23,31,74,299,10,3,0,136,3,25,8,1,32.46
1,Robert Griffin III,QB,1,WAS,NO,19,26,73,320,12,2,0,140,9,42,5,0,25.00
2,Tony Romo,QB,1,DAL,NYG,22,29,76,307,11,3,1,130,5,12,2,0,23.48
3,Aaron Rodgers,QB,1,GB,SF,30,44,68,303,7,2,1,93,5,27,5,0,22.82
4,Drew Brees,QB,1,NO,WAS,24,52,46,339,7,3,2,71,0,0,0,0,21.56


In [5]:
raw_qb_2012 = raw_qb_2012.rename(columns={'Att.1':'rush_att',
                       'Yds.1':'rush_yds',
                       'Yds/Att.1':'rush_yd/att',
                       'TD.1':'rush_td'})

In [113]:
weekly_stats = raw_qb_2012.groupby(['Player', 'Week']).sum()

In [6]:
team_stats = raw_qb_2012.groupby(['Player','Week','Opp']).sum()

In [7]:
team_stats.head(), team_stats.shape

(                        Comp  Att  Pct  Yds  Yds/Att  TD  Int  QB Rating  \
 Player        Week Opp                                                     
 Aaron Rodgers 1    SF     30   44   68  303        7   2    1         93   
               2    CHI    22   32   69  219        7   1    1         85   
               3    SEA    26   39   67  223        6   0    0         81   
               4    NO     31   41   76  319        8   4    1        120   
               5    IND    20   32   63  235        7   3    1        103   
 
                         rush_att  rush_yds  rush_yd/att  rush_td  \
 Player        Week Opp                                             
 Aaron Rodgers 1    SF          5        27            5        0   
               2    CHI         3        -6           -2        0   
               3    SEA         2        17            9        0   
               4    NO          5        13            3        0   
               5    IND         5        57  

In [10]:
weekly_stats.head(), weekly_stats.shape

(                    Comp  Att  Pct  Yds  Yds/Att  TD  Int  QB Rating  \
 Player        Week                                                     
 Aaron Rodgers 1       30   44   68  303        7   2    1         93   
               2       22   32   69  219        7   1    1         85   
               3       26   39   67  223        6   0    0         81   
               4       31   41   76  319        8   4    1        120   
               5       20   32   63  235        7   3    1        103   
 
                     rush_att  rush_yds  rush_yd/att  rush_td  Fantasy Points  
 Player        Week                                                            
 Aaron Rodgers 1            5        27            5        0           22.82  
               2            3        -6           -2        0           10.16  
               3            2        17            9        0           10.62  
               4            5        13            3        0           28.06  
       

In [11]:
y=weekly_stats['Fantasy Points']

In [8]:
# Define a dataframe to contain the current year 3 game moving averages of all 2012 QBs
def create_recent_dataframe(df):
    columns = []
    for col in df.columns:
        columns.append('{}_ma'.format(col.lower()))
    new_df = pd.DataFrame(columns=columns, index =df.index)
    return new_df

In [7]:
# Define a dataframe to contain the career averages for all 2012 QBs
def create_career_dataframe(df):
    columns = []
    for col in df.columns:
        columns.append('{}_car'.format(col.lower()))
    new_df = pd.DataFrame(columns=columns, index =df.index)
    return new_df

In [9]:
# Create recent and current dataframes for all 2012 QBs
qb_recent_2012= create_recent_dataframe(team_stats)

In [10]:
qb_recent_2012.head()

comp_ma att_ma pct_ma yds_ma yds/att_ma td_ma int_ma  \
Player        Week Opp                                                        
Aaron Rodgers 1    SF      NaN    NaN    NaN    NaN        NaN   NaN    NaN   
              2    CHI     NaN    NaN    NaN    NaN        NaN   NaN    NaN   
              3    SEA     NaN    NaN    NaN    NaN        NaN   NaN    NaN   
              4    NO      NaN    NaN    NaN    NaN        NaN   NaN    NaN   
              5    IND     NaN    NaN    NaN    NaN        NaN   NaN    NaN   

                       qb rating_ma rush_att_ma rush_yds_ma rush_yd/att_ma  \
Player        Week Opp                                                       
Aaron Rodgers 1    SF           NaN         NaN         NaN            NaN   
              2    CHI          NaN         NaN         NaN            NaN   
              3    SEA          NaN         NaN         NaN            NaN   
              4    NO           NaN         NaN         NaN            NaN   
              5    IND          NaN         NaN         NaN            NaN   

                       rush_td_ma fantasy points_ma  
Player        Week Opp                               
Aaron Rodgers 1    SF         NaN               NaN  
              2    CHI        NaN               NaN  
              3    SEA        NaN               NaN  
              4    NO         NaN               NaN  
              5    IND        NaN               NaN

In [92]:
team_stats.loc['Aaron Rodgers'].index.levels[0]

Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], dtype='int64', name='Week')

In [73]:
for col in team_stats:  # Iterate through every attribute in the team_stats dataframe
    ma3 = []  # Create an empty list to hold the 3 game moving average of the current attribute
    for player in team_stats.index.levels[0]:  # Iterate through every player in the team_stats df
        total = [] # create an empty list to hold the weekly attributes value
        for week in team_stats.loc[player].index.levels[0]: # Iterate throough every week for the current player
            if week <= 3:
                ma3.append(0)
            else:
                ma3.append(np.mean((team_stats.loc[player].loc[week][col]),(team_stats.loc[player].loc[week-1][col]),(team_stats.loc[player].loc[week-2][col])))  # Add the average of the total list to the season average list
    qb_recent_2012['{}_ma'.format(col.lower())] = ma3 # Insert season average list for current att

TypeError: unsupported operand type(s) for -: 'tuple' and 'int'

In [71]:
len(ma3)

629

In [72]:
qb_recent_2012.tail()

comp_ma  att_ma  pct_ma  yds_ma  yds/att_ma  td_ma  \
Player       Week Opp                                                       
Tyrod Taylor 4    CLE        0       0       0       0           0      0   
             5    KC         0       0       0       0           0      0   
             9    CLE        0       0       0       0           0      0   
             10   OAK        0       0       0       0           0      0   
             17   CIN        0       0       0       0           0      0   

                       int_ma  qb rating_ma  rush_att_ma  rush_yds_ma  \
Player       Week Opp                                                   
Tyrod Taylor 4    CLE       0             0            0            0   
             5    KC        0             0            0            0   
             9    CLE       0             0            0            0   
             10   OAK       0             0            0            0   
             17   CIN       0             0            0            0   

                       rush_yd/att_ma  rush_td_ma  fantasy points_ma  
Player       Week Opp                                                 
Tyrod Taylor 4    CLE               0           0                  0  
             5    KC                0           0                  0  
             9    CLE               0           0                  0  
             10   OAK               0           0                  0  
             17   CIN               0           0                  0

In [95]:
for col in team_stats:  # Iterate through every attribute in the team_stats dataframe
    ma3 = []  # Create an empty list to hold the 3 game moving average of the current attribute
    for player in team_stats.index.levels[0]:  # Iterate through every player in the team_stats df
        total = [] # create an empty list to hold the weekly attributes value
        for week in team_stats.loc[player].index.levels[0]:
            if week > 1:
                print(team_stats.loc[player].loc[week-1][col])

Opp
SF    30
Name: Comp, dtype: int64
Opp
CHI    22
Name: Comp, dtype: int64
Opp
SEA    26
Name: Comp, dtype: int64
Opp
NO    31
Name: Comp, dtype: int64
Opp
IND    20
Name: Comp, dtype: int64
Opp
HOU    24
Name: Comp, dtype: int64
Opp
LAR    30
Name: Comp, dtype: int64
Opp
JAX    22
Name: Comp, dtype: int64
Opp
ARI    14
Name: Comp, dtype: int64
Series([], Name: Comp, dtype: int64)
Opp
DET    19
Name: Comp, dtype: int64
Opp
NYG    14
Name: Comp, dtype: int64
Opp
MIN    27
Name: Comp, dtype: int64
Opp
DET    14
Name: Comp, dtype: int64
Opp
CHI    23
Name: Comp, dtype: int64
Opp
TEN    27
Name: Comp, dtype: int64
Opp
GB    20
Name: Comp, dtype: int64
Opp
DET    20
Name: Comp, dtype: int64
Opp
MIN    24
Name: Comp, dtype: int64
Opp
NYJ    12
Name: Comp, dtype: int64
Opp
BUF    18
Name: Comp, dtype: int64
Opp
NYG    19
Name: Comp, dtype: int64
Opp
SEA    14
Name: Comp, dtype: int64
Opp
ARI    18
Name: Comp, dtype: int64
Series([], Name: Comp, dtype: int64)
Opp
LAR    7
Name: Comp, dtype: 

Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Opp
PIT    0
Name: Comp, dtype: int64
Series([], Name: Comp, dtype: int64)
Opp
KC    0
Name: Comp, dtype: int64
Series([], Name: Comp, dtype: int64)
Opp
DEN    9
Name: Comp, dtype: int64
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)

Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Opp
BAL    2
Name: Comp, dtype: int64
Opp
CLE    26
Name: Comp, dtype: int64
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Opp
CHI    9
Name: Comp, dtype: int64
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int64)
Series([], Name: Comp, dtype: int6

Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Opp
KC    14
Name: Att, dtype: int64
Opp
BAL    39
Name: Att, dtype: int64
Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Opp
TB    33
Name: Att, d

Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Series([], Name: Att, dtype: int64)
Opp
DAL    32
Name: Att, dtype: int64
Opp
WAS    46
Name: Att, dtype: int64
Opp
CAR    21
Name: Att, dtype: int64
Opp
DAL    34
Name: Att, dtype: int64
Opp
TB    51
Name: Att, dtype: int64
Opp
CIN    33
Name: Att, dtype: int64
Opp
WAS    48
Name: Att, dtype: int64
Opp
PIT    26
Name: Att, dtype: int64
Opp
ATL    37
Name: Att, dtype: int64
Opp
HOU    52
Name: Att, dtype: int64
Opp
OAK    38
Name: Att, dtype: int64
Opp
NE    44
Name: Att, dtype: int64
Opp
LAC    30
Name: Att, dtype: int64
Series([], Name: Att, dtype: int64)
Opp
NO    30
Name: Att, dtype: int64
Opp
CIN    35
Name: Att, dtype: int64
Opp
CAR    38
Name: Att, dtype: int64
Opp
LAC    42
Name: Att, dtype: int

...

Opp
DEN    0
Name: rush_yds, dtype: int64
Series([], Name: rush_yds, dtype: int64)
Opp
CLE    0
Name: rush_yds, dtype: int64
Opp
KC   -2
Name: rush_yds, dtype: int64
Opp
TB    5
Name: rush_yds, dtype: int64
Opp
DEN    0
Name: rush_yds, dtype: int64
Opp
BAL    0
Name: rush_yds, dtype: int64
Opp
CIN    20
Name: rush_yds, dtype: int64
Opp
PIT    3
Name: rush_yds, dtype: int64
Opp
CAR    4
Name: rush_yds, dtype: int64
Opp
NYJ    4
Name: rush_yds, dtype: int64
Opp
NO    42
Name: rush_yds, dtype: int64
Opp
LAR    82
Name: rush_yds, dtype: int64
Opp
CIN    74
Name: rush_yds, dtype: int64
Opp
TB    36
Name: rush_yds, dtype: int64
Opp
ATL    7
Name: rush_yds, dtype: int64
Opp
MIN    138
Name: rush_yds, dtype: int64
Opp
NYG    89
Name: rush_yds, dtype: int64
Opp
PIT    8
Name: rush_yds, dtype: int64
Opp
CAR    53
Name: rush_yds, dtype: int64
Series([], Name: rush_yds, dtype: int64)
Opp
PHI    84
Name: rush_yds, dtype: int64
Opp
DAL    29
Name: rush_yds, dtype: int64
Opp
NYG    72
Name: rush_yds,

Opp
DAL    0
Name: rush_yd/att, dtype: int64
Opp
TB   -1
Name: rush_yd/att, dtype: int64
Opp
CAR    3
Name: rush_yd/att, dtype: int64
Opp
PHI    0
Name: rush_yd/att, dtype: int64
Opp
CLE   -1
Name: rush_yd/att, dtype: int64
Opp
SF   -1
Name: rush_yd/att, dtype: int64
Opp
WAS    2
Name: rush_yd/att, dtype: int64
Opp
DAL    3
Name: rush_yd/att, dtype: int64
Opp
PIT    0
Name: rush_yd/att, dtype: int64
Opp
CIN    1
Name: rush_yd/att, dtype: int64
Series([], Name: rush_yd/att, dtype: int64)
Opp
GB    6
Name: rush_yd/att, dtype: int64
Opp
WAS    5
Name: rush_yd/att, dtype: int64
Opp
NO   -2
Name: rush_yd/att, dtype: int64
Opp
ATL    0
Name: rush_yd/att, dtype: int64
Opp
BAL    8
Name: rush_yd/att, dtype: int64
Series([], Name: rush_yd/att, dtype: int64)
Series([], Name: rush_yd/att, dtype: int64)
Series([], Name: rush_yd/att, dtype: int64)
Opp
NO    0
Name: rush_yd/att, dtype: int64
Series([], Name: rush_yd/att, dtype: int64)
Opp
HOU    0
Name: rush_yd/att, dtype: int64
Series([], Name: rus

Name: rush_td, dtype: int64
Opp
IND    0
Name: rush_td, dtype: int64
Opp
HOU    0
Name: rush_td, dtype: int64
Series([], Name: rush_td, dtype: int64)
Series([], Name: rush_td, dtype: int64)
Series([], Name: rush_td, dtype: int64)
Series([], Name: rush_td, dtype: int64)
Series([], Name: rush_td, dtype: int64)
Series([], Name: rush_td, dtype: int64)
Series([], Name: rush_td, dtype: int64)
Series([], Name: rush_td, dtype: int64)
Series([], Name: rush_td, dtype: int64)
Opp
BAL    0
Name: rush_td, dtype: int64
Opp
TB    0
Name: rush_td, dtype: int64
Series([], Name: rush_td, dtype: int64)
Opp
OAK    0
Name: rush_td, dtype: int64
Series([], Name: rush_td, dtype: int64)
Series([], Name: rush_td, dtype: int64)
Opp
CIN    0
Name: rush_td, dtype: int64
Opp
DEN    0
Name: rush_td, dtype: int64
Opp
CAR    0
Name: rush_td, dtype: int64
Opp
CLE    0
Name: rush_td, dtype: int64
Opp
OAK    0
Name: rush_td, dtype: int64
Opp
IND    0
Name: rush_td, dtype: int64
Opp
PHI    0
Name: rush_td, dtype: int64
O

Series([], Name: rush_td, dtype: int64)
Series([], Name: rush_td, dtype: int64)
Series([], Name: rush_td, dtype: int64)
Series([], Name: rush_td, dtype: int64)
Series([], Name: rush_td, dtype: int64)
Series([], Name: rush_td, dtype: int64)
Series([], Name: rush_td, dtype: int64)
Series([], Name: rush_td, dtype: int64)
Opp
BAL    0
Name: rush_td, dtype: int64
Series([], Name: rush_td, dtype: int64)
Series([], Name: rush_td, dtype: int64)
Series([], Name: rush_td, dtype: int64)
Series([], Name: rush_td, dtype: int64)
Series([], Name: rush_td, dtype: int64)
Opp
CAR    0
Name: rush_td, dtype: int64
Series([], Name: rush_td, dtype: int64)
Series([], Name: rush_td, dtype: int64)
Series([], Name: rush_td, dtype: int64)
Series([], Name: rush_td, dtype: int64)
Series([], Name: rush_td, dtype: int64)
Series([], Name: rush_td, dtype: int64)
Series([], Name: rush_td, dtype: int64)
Opp
NYJ    0
Name: rush_td, dtype: int64
Series([], Name: rush_td, dtype: int64)
Opp
TEN    0
Name: rush_td, dtype: in

Name: Fantasy Points, dtype: float64
Opp
HOU    6.58
Name: Fantasy Points, dtype: float64
Series([], Name: Fantasy Points, dtype: float64)
Opp
CLE    12.22
Name: Fantasy Points, dtype: float64
Opp
OAK    29.74
Name: Fantasy Points, dtype: float64
Opp
PIT    6.46
Name: Fantasy Points, dtype: float64
Opp
LAC    17.8
Name: Fantasy Points, dtype: float64
Opp
PIT    7.42
Name: Fantasy Points, dtype: float64
Opp
WAS    16.18
Name: Fantasy Points, dtype: float64
Opp
DEN    13.96
Name: Fantasy Points, dtype: float64
Opp
NYG    25.36
Name: Fantasy Points, dtype: float64
Series([], Name: Fantasy Points, dtype: float64)
Series([], Name: Fantasy Points, dtype: float64)
Series([], Name: Fantasy Points, dtype: float64)
Series([], Name: Fantasy Points, dtype: float64)
Opp
TEN   -0.1
Name: Fantasy Points, dtype: float64
Series([], Name: Fantasy Points, dtype: float64)
Series([], Name: Fantasy Points, dtype: float64)
Series([], Name: Fantasy Points, dtype: float64)
Series([], Name: Fantasy Points, dtyp

In [37]:
qb_recent_2012.head()

comp_ma     att_ma  pct_ma      yds_ma  yds/att_ma  \
Player        Week Opp                                                       
Aaron Rodgers 1    SF     30.00  44.000000    68.0  303.000000    7.000000   
              2    CHI    26.00  38.000000    68.5  261.000000    7.000000   
              3    SEA    26.00  38.333333    68.0  248.333333    6.666667   
              4    NO     27.25  39.000000    70.0  266.000000    7.000000   
              5    IND    25.80  37.600000    68.6  259.800000    7.000000   

                        td_ma    int_ma  qb rating_ma  rush_att_ma  \
Player        Week Opp                                               
Aaron Rodgers 1    SF    2.00  1.000000     93.000000     5.000000   
              2    CHI   1.50  1.000000     89.000000     4.000000   
              3    SEA   1.00  0.666667     86.333333     3.333333   
              4    NO    1.75  0.750000     94.750000     3.750000   
              5    IND   2.00  0.800000     96.400000     4.000000   

                        rush_yds_ma  rush_yd/att_ma  rush_td_ma  \
Player        Week Opp                                            
Aaron Rodgers 1    SF     27.000000            5.00         0.0   
              2    CHI    10.500000            1.50         0.0   
              3    SEA    12.666667            4.00         0.0   
              4    NO     12.750000            3.75         0.0   
              5    IND    21.600000            5.20         0.0   

                        fantasy points_ma  
Player        Week Opp                     
Aaron Rodgers 1    SF           22.820000  
              2    CHI          16.490000  
              3    SEA          14.533333  
              4    NO           17.915000  
              5    IND          19.352000

In [118]:
# Populate recent dataframe with current year 3 game moving-averages of players
for col in weekly_stats:    # Iterate through every attribute in the weekly_stats dataframe
    ma3 = []        # Create an empty list to hold the current attribute's 3 game moving average                    
    for player in weekly_stats.index.levels[0]: # Iterate through every player in the weekly_stats df
        total = [] # Create an empty list to hold the weekly attributes value
        for week in weekly_stats.index.levels[1]: # Iterate through every week for the current player
            games = 0 # Set count for 3 game moving average 
            games_lst = [] # Create a list to hold 3 game block 
            if week < 4: # 3 game moving average does not instantiate until game 4
                games_lst.append(weekly_stats.loc[player].loc[week][col]) # Add current week value to 3 game block
                ma3.append(0) # Until game 4, append moving average for current attribute to 0
                games += 1
            else:
                if games < 3: # Check value of count for 3 game block
                    try: # Check for team bye week
                        games_lst.append(weekly_stats.loc[player].loc[week][col])  # Add current week value to 3 game block
                        games += 1 # Incease count
                    except: # Pass if team's bye week
                        pass
                ma3.append(round(np.mean(games_lst),2))  # Add average of three game block to 3 game moving average list
        
    qb_current_2012['{}_ma'.format(col.lower())] = ma3 # Insert 3 game moving average list for current attribute into the current stats dataframe

           

C:\Users\bobal\Anaconda2\envs\py36\lib\site-packages\numpy\core\fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\bobal\Anaconda2\envs\py36\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


KeyError: 'the label [1] is not in the [index]'

In [123]:
weekly_stats.head()

Comp  Att  Pct  Yds  Yds/Att  TD  Int  QB Rating  \
Player        Week                                                     
Aaron Rodgers 1       30   44   68  303        7   2    1         93   
              2       22   32   69  219        7   1    1         85   
              3       26   39   67  223        6   0    0         81   
              4       31   41   76  319        8   4    1        120   
              5       20   32   63  235        7   3    1        103   

                    rush_att  rush_yds  rush_yd/att  rush_td  Fantasy Points  
Player        Week                                                            
Aaron Rodgers 1            5        27            5        0           22.82  
              2            3        -6           -2        0           10.16  
              3            2        17            9        0           10.62  
              4            5        13            3        0           28.06  
              5            5        57           11        0           25.10

In [126]:
weekly_stats.loc['Aaron Rodgers'].loc[3]['Comp'] # Add current week value to 3 game block


26.0

In [73]:
len(ma3)

1173

In [58]:
qb_recent_2012.head()

AttributeError: 'list' object has no attribute 'head'

In [75]:
df = pd.read_csv('qb_recent_2012.csv')
df.head()

,Player,Week,comp_ma,att_ma,pct_ma,yds_ma,yds/att_ma,td_ma,int_ma,qb rating_ma,rush_att_ma,rush_yds_ma,rush_yd/att_ma,rush_td_ma,fantasy points_ma
0,Aaron Rodgers,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
1,Aaron Rodgers,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
2,Aaron Rodgers,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
3,Aaron Rodgers,4,31.0,41.0,76.0,319.0,8.0,4.0,1.0,120.0,5.0,13.0,3.0,0.0,28.06
4,Aaron Rodgers,5,20.0,32.0,63.0,235.0,7.0,3.0,1.0,103.0,5.0,57.0,11.0,0.0,25.10


In [76]:
df.shape

(629, 15)

In [163]:
team_stats.loc['Aaron Rodgers'].loc[1]['Comp']

Opp
SF    30
Name: Comp, dtype: int64

In [156]:
len(ma3), qb_recent_2012.shape

(1173, (629, 13))

In [154]:
qb_recent_2012['{}_ma'.format(col.lower())] = ma3       

C:\Users\bobal\Anaconda2\envs\py36\lib\site-packages\numpy\core\fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\bobal\Anaconda2\envs\py36\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ValueError: Length of values does not match length of index

In [ ]:
team_.loc[]

In [96]:
for col in team_stats.columns:
    ma3 = []
    for player in team_stats.index.levels[0]:
        for week in team_stats.index.levels[1]:
            if week <= 3:
                ma3.append(0)
            else:
                games = 0
                games_lst = []
                if games <= 3:
                    try:
                        games_lst.append(team_stats.loc[player].loc[week][col])
                        games += 1
                    except:
                        pass
                        games += 1
                ma3.append(np.mean(games_lst))
    

C:\Users\bobal\Anaconda2\envs\py36\lib\site-packages\numpy\core\fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\bobal\Anaconda2\envs\py36\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [97]:
len(ma3)

1173

In [142]:
qb_recent_2012['{}_ma'.format(col.lower())]=ma3
                
                    
                    

C:\Users\bobal\Anaconda2\envs\py36\lib\site-packages\numpy\core\fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\bobal\Anaconda2\envs\py36\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ValueError: Length of values does not match length of index

In [33]:
len(ma)

966

In [35]:
qb_recent_2012.shape

(629, 13)